In [25]:
from tensorflow import keras

Prepare the dataset

In [26]:
train_dir = 'data/split/train'
val_dir = 'data/split/test'

The RGB channel value is in the range [0, 255] . This is not ideal for neural networks; in general you should find a way to reduce your input value.

In [27]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [28]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [29]:
print(train_generator)

Create network

In [30]:
from keras import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D

In [31]:
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.1))
emotion_model.add(Dense(7, activation='softmax'))

Compile and Train

In [32]:
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])

emotion_model_info = emotion_model.fit(
        train_generator,
        epochs=50,
        batch_size=64,
        validation_data=validation_generator)

Epoch 1/50


/home/tvc/miniconda3/envs/ml/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


  3/449 [..............................] - ETA: 12s - loss: 1.9064 - accuracy: 0.2500 

2022-12-26 03:30:41.233276: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_3/dropout_9/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


449/449 [==============================] - 13s 27ms/step - loss: 1.7563 - accuracy: 0.2833 - val_loss: 1.6317 - val_accuracy: 0.3819
Epoch 2/50
449/449 [==============================] - 11s 25ms/step - loss: 1.5696 - accuracy: 0.3975 - val_loss: 1.4999 - val_accuracy: 0.4313
Epoch 3/50
449/449 [==============================] - 13s 30ms/step - loss: 1.4808 - accuracy: 0.4355 - val_loss: 1.4351 - val_accuracy: 0.4565
Epoch 4/50
449/449 [==============================] - 14s 30ms/step - loss: 1.4092 - accuracy: 0.4664 - val_loss: 1.3607 - val_accuracy: 0.4886
Epoch 5/50
449/449 [==============================] - 14s 31ms/step - loss: 1.3493 - accuracy: 0.4875 - val_loss: 1.3368 - val_accuracy: 0.4979
Epoch 6/50
449/449 [==============================] - 14s 31ms/step - loss: 1.2951 - accuracy: 0.5108 - val_loss: 1.2837 - val_accuracy: 0.5116
Epoch 7/50
449/449 [==============================] - 14s 31ms/step - loss: 1.2489 - accuracy: 0.5288 - val_loss: 1.2615 - val_accuracy: 0.5199
Epo

Save model

In [33]:
emotion_model.save('models/model.h5')